## User Inputs

In [ ]:
# If using a Mac, need to specify path that contains chromedriver
driver_path = "/Users/Matthew/Documents/Consulting/Lectures/Webscraping/chromedriver"
year = '1973'

## Libraries

In [ ]:
from urllib.request import urlopen
from bs4 import BeautifulSoup
from selenium import webdriver
import pandas as pd
import time
import warnings
import numpy as np
warnings.filterwarnings("ignore")

## Functions

In [ ]:
def add_data(box,tables,table):
    count = 1
    header = tables[-3].findAll("tr")[1].text.split('\n')
    header.pop(0)
    header.pop(-1)
    for row in table.findAll("tr"):
        length = len(table.findAll("tr"))
        if count == length:
            try:
                name = row.find_all('a')[0].text = 'Total'
            except:
                pass
        else:
            try:
                name = row.find_all('a')[0].text
            except:
                pass
        try:
            cells = row.findAll("td")
            cells = [ele.text for ele in cells]
            if len(cells)>1:
                cells.insert(0, name)
                box.append(cells)
        except:
            pass
        count += 1
    return header
        
def create_minutes(box):
    box['Minutes']=box['MP'].str.split(":", expand=True).fillna(0)[0].apply(pd.to_numeric)
    box['Seconds']=box['MP'].str.split(":", expand=True).fillna(0)[1].apply(pd.to_numeric)
    box['_MP']=box['Minutes']+(box['Seconds']/60)
    box['_MP']=box.loc[:,'_MP':].div(box.iloc[-1]['_MP':]/5)
    box=box[['Player','_MP']]
    box=box.iloc[:-1]
    box = box.T.reset_index()
    box.columns = box.iloc[0]
    box = box[1:]
    box=box.drop('Player',axis=1)
    box.fillna(0,inplace=True)
    return box

def get_roster(roster_url):
    ##Connect to a website using a proxy server
    chrome_options = webdriver.ChromeOptions()
    #chrome_options.add_argument('--proxy-server=%s' % PROXY)
    chrome_options.add_argument('--headless')

    # If not using Mac
    browser = webdriver.Chrome(executable_path = driver_path, chrome_options=chrome_options)
    browser.get(roster_url)

    ##Grab the raw webpage
    innerHTML = browser.execute_script("return document.body.innerHTML")
    soup = BeautifulSoup(innerHTML,"html.parser")
    tables = soup.findAll("table")

    players = []
    for row in tables[0].findAll("tr"):
        try:
            player = row.find('a').text
            players.append(player)
        except:
            pass
    return players
        
def get_box_scores(games,homes,differentials):
    boxs = []
    game_count = 1
    for game,home,diff in zip(games,homes,differentials):
        browser.get(game)
        innerHTML = browser.execute_script("return document.body.innerHTML")
        soup = BeautifulSoup(innerHTML,"html.parser")
        tables = soup.findAll("table")

        box = []
        box2 = []
        table_count = 1
        try:
            team_check = browser.find_element_by_id('content').text.split('\n')[0].split('at')[0].strip()
        except:
            team_check = browser.find_element_by_id('content').text.split('\n')[0].split('vs')[0].strip()
        for table in tables[-3:]:
            if table_count == 1:
                if team_check==team:
                    header=add_data(box,tables,table)
                else:
                    header=add_data(box2,tables,table)
            else:
                if team_check==team:
                    header=add_data(box2,tables,table)
                else: 
                    header=add_data(box,tables,table)
            table_count += 1
        try:
            box = pd.DataFrame(box, columns = header)
            box2 = pd.DataFrame(box2, columns = header)
            box=create_minutes(box)
            box2=create_minutes(box2)
            box['Differential'] = diff
            box['Location']=home
            box = pd.concat([box, box2], axis=1)
            if len(boxs)==0:
                boxs=box.copy()
            else:
                boxs=pd.concat([boxs, box], axis=0)
            print(game_count,home,diff)
        except Exception as e:
            print('cannot pull game')
        game_count+=1
        time.sleep(1)
    return boxs

def get_games(url):
    browser.get(url)
    ##Grab the raw webpage
    innerHTML = browser.execute_script("return document.body.innerHTML")
    soup = BeautifulSoup(innerHTML,"html.parser")
    tables = soup.findAll("table")

    games = []
    homes = []
    differentials = []
    try:
        tbl = tables[-2].findAll("tr")
    except:
        tbl = tables[0].findAll("tr")
    for row in tbl:
        td = row.findAll("td")
        try:
            game = "https://www.basketball-reference.com"+td[2].find_all('a')[0].get('href')
            diff=float(td[7].text)-float(td[8].text)
            differentials.append(diff)
            if td[3].text:
                homes.append(td[3].text)
            else:
                homes.append('H')
            games.append(game)
        except:
            pass
    print(len(games))
    return games,homes,differentials

def rename_roster(box,roster):
    global roster_t
    try:
        box[roster]
        roster2=roster.copy()
    except Exception as e:
        error = str(e).split(' not ')[0].split('"')[1]
        print(error)
        roster2 = [x for x in roster if x not in str(e).split(' not ')[0]]
    roster_t = []
    for r in roster2:
        roster_t.append(r + "_T")
    col_rename_dict = {i:j for i,j in zip(roster2,roster_t)}
    print(col_rename_dict)
    box2 = box.rename(columns=col_rename_dict)
    return box2

## Scrape Teams

In [ ]:
# Get all teams in a list and their links
url1 = 'https://www.basketball-reference.com/leagues/NBA_'+year+'.html'

##Connect to a website using a proxy server
chrome_options = webdriver.ChromeOptions()
#chrome_options.add_argument('--proxy-server=%s' % PROXY)
chrome_options.add_argument('--headless')

# If not using Mac
browser = webdriver.Chrome(executable_path = driver_path, chrome_options=chrome_options)
browser.get(url1)

##Grab the raw webpage
innerHTML = browser.execute_script("return document.body.innerHTML")
soup = BeautifulSoup(innerHTML,"html.parser")
tables = soup.findAll("table")

teams =[]
teamz = []
for row in tables[0].findAll("tr"):
    try:
        a = row.find_all('a', href=True)
        for b in a:
            teamz.append(b['href'])
            teams.append(b.text)
    except:
        pass
teamz

## Scrape Box Scores

In [ ]:
##Connect to a website using a proxy server
chrome_options = webdriver.ChromeOptions()
#chrome_options.add_argument('--proxy-server=%s' % PROXY)
chrome_options.add_argument('--headless')

boxs = pd.DataFrame()
# If not using Mac
browser = webdriver.Chrome(executable_path = driver_path, chrome_options=chrome_options)

for team,name in zip(teams,teamz):
    try:
        print(team)
        url = 'https://www.basketball-reference.com'+name.split('.html')[0]+'_games.html'
        roster_url = 'https://www.basketball-reference.com'+name

        roster = get_roster(roster_url)
        games,homes,differentials = get_games(url)
        team_box = get_box_scores(games,homes,differentials)
        team_box2 = rename_roster(team_box,roster)
        if len(boxs)==0:
            boxs=team_box2.copy()
        else:
            boxs=pd.concat([boxs, team_box2], axis=0)
        print(len(boxs))
    except:
        pass

In [ ]:
boxs.to_excel('BoxScores/'+str(int(year))+'box.xlsx')